<a href="https://colab.research.google.com/github/Last-Vega/Klis_Workshop_MachineLearning/blob/master/ML8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# torchtextというパッケージをアップデートしておく（古いバージョンの場合は後で出現する単語埋め込みが読み込めない）
!pip install -U torchtext==0.7.0

In [2]:
# 分かち書きを行うためにjanomeというPythonパッケージをインストール
!pip install janome

In [3]:
# ファイルのダウンロード
!wget "https://drive.google.com/uc?export=download&id=1c0tXuRt2GE8szurDI01P0YkbgfEuNi6o" -O twitterJSA_data.csv

--2020-10-31 22:45:32--  https://drive.google.com/uc?export=download&id=1c0tXuRt2GE8szurDI01P0YkbgfEuNi6o
Resolving drive.google.com (drive.google.com)... 74.125.20.102, 74.125.20.101, 74.125.20.113, ...
Connecting to drive.google.com (drive.google.com)|74.125.20.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-58-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ehopv8k0keu1oi7bd5t1ajpr8ncq7vu9/1604184300000/07803272131756145988/*/1c0tXuRt2GE8szurDI01P0YkbgfEuNi6o?e=download [following]
--2020-10-31 22:45:32--  https://doc-10-58-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ehopv8k0keu1oi7bd5t1ajpr8ncq7vu9/1604184300000/07803272131756145988/*/1c0tXuRt2GE8szurDI01P0YkbgfEuNi6o?e=download
Resolving doc-10-58-docs.googleusercontent.com (doc-10-58-docs.googleusercontent.com)... 74.125.195.132, 2607:f8b0:400e:c09::84
Connecting to doc-10-58-docs.googleusercontent.com (doc-10-58-

In [4]:
from torchtext import data
from janome.tokenizer import Tokenizer # janomeからTokenizerクラスをimportする

# janomeはPythonで書かれた形態素解析器であり単語の分かち書きなどが可能
tokenizer = Tokenizer(wakati=True)

def tokenize(text):
    # 単語の分かち書きを行う
    return list(tokenizer.tokenize(text))

# Fieldオブジェクトは読み込んだデータの各項目（CSVの場合は各列）をどのように処理するかを決定する
# Fieldの引数に何も指定しない場合には，テキストのような系列データと仮定し指定されたtokenizerで処理される
# 今回用いるファイル中のidやlabelのように，系列データでなく，tokenizeする必要もないデータの場合は
# sequential=False, use_vocab=Falseを指定する
ID = data.Field(sequential=False, use_vocab=False)
LABEL = data.Field(sequential=False, use_vocab=False)
TEXT = data.Field(tokenize=tokenize) # tokenize引数に指定した関数でテキストを処理する

# CSVファイルを読み込み
dataset = data.TabularDataset(
    path='./twitterJSA_data.csv', # 読み込みファイル
    format='csv', # 読み込むファイルの形式
    fields=[('id', ID), ('label', LABEL), ('text', TEXT)], # 各列ごとにFieldオブジェクトを設定
    skip_header=True # 最初の行は各列の見出しなので読み込まない
    )

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes f

In [5]:
from torch.utils.data import random_split

train_size = 20000 # 先頭の20,000件を訓練データとして用いることにする
batch_size = 32 # ミニバッチのサイズを設定

# 訓練データとテストデータに分割
train_dataset, test_dataset = dataset.split(split_ratio=train_size/len(dataset))

# 訓練データを読み込むためのイテレータを準備
train_iterator = data.BucketIterator(train_dataset, batch_size=batch_size, train=True)
test_iterator = data.BucketIterator(test_dataset, batch_size=batch_size, train=False, sort=False)

/usr/local/lib/python3.6/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [6]:
# TEXTフィールド中の単語を収集して，単語の種類数や頻度などを計算し，これをVocabクラスのオブジェクトしてまとめて，
# TEXTフィールドのデータ属性として保存する
TEXT.build_vocab(train_dataset, min_freq=5) # `min_freq=5`: 頻度が5回未満の単語は無視する設定

In [7]:
from torchtext.vocab import FastText

# FastTextの単語埋め込みを設定し，TEXT.vocabを更新
# 単語埋め込みをダウンロードする必要があるため少し時間がかかる．
TEXT.build_vocab(train_dataset, min_freq=5, vectors=FastText(language="ja"))

.vector_cache/wiki.ja.vec: 1.37GB [00:26, 52.8MB/s]                            
100%|█████████▉| 579756/580000 [01:01<00:00, 9546.73it/s]

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TextClassifier(nn.Module):
    def __init__(self, vocab):
        super(TextClassifier, self).__init__()

        # 単語埋め込み（FastText）を設定する
        self.embedding = torch.nn.Embedding.from_pretrained(
            embeddings=vocab.vectors, freeze=False)
        
        emb_size = vocab.vectors.shape[1]
        self.lstm = nn.LSTM(emb_size, 100) # LSTMを用意する
        self.fc = nn.Linear(100, 1) # 1 x 100 の行列（この場合はベクトル）を含む全結合層を設定

    def forward(self, seq):
        x = self.embedding(seq) # 各単語を単語埋め込みに変換する
        output, (h_n, c_n) = self.lstm(x) # LSTMを適用する
        h = h_n.view(-1, 100) # 各系列の最後の入力（単語）に対応する隠れ状態は，1 x バッチサイズ x 隠れ状態の次元数，となっているため，これを，バッチサイズ x 隠れ状態の次元数，と変換する．
        y = self.fc(F.relu(h)) # ReLUをかけてから，全結合層に通す．
        y = y.squeeze() # yは バッチサイズ x 1 という行列になっているため，バッチサイズと同じ次元数を持つベクトルに変換
        return y

In [9]:
import torch
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # CPUもしくはGPUのどちらを使うかを設定
model = TextClassifier(TEXT.vocab) # ニューラルネットワークモデルのインスタンスを生成
model = model.to(device) # CPUもしくはGPUのどちらを設定
optimizer = optim.Adam(model.parameters()) # 基本的な学習方法はミニバッチ勾配降下法ではあるが，その中でもよく用いられるAdamと呼ばれる方法を用いることにする

criterion = nn.BCEWithLogitsLoss() # 二値分類用の交差エントロピーを最小化することにする

epoch_size = 20 # 勾配降下法はすべてのデータでパラメータを更新したら終わりではなく，全データでの更新（=1エポック）を複数回行う必要がある

model.train() # モデルを学習モードに変更

# `epoch_size`の数だけ以下を繰り返す
for epoch in range(epoch_size):
    losses = []
    # イテレータはミニバッチ勾配降下法のために，`batch_size`で指定した数ごとにデータをわけて読み込んでくれる．
    for batch_idx, batch in enumerate(train_iterator):
        texts, labels = batch.text.to(device), batch.label.to(device)
        optimizer.zero_grad() # 勾配の初期化
        y = model(texts) # 現時点でのモデルの出力を得る
        loss = criterion(y, labels.type(torch.float)) # 交差エントロピーの計算
        loss.backward() # 交差エントロピーの勾配計算
        optimizer.step() # パラメータ更新
        losses.append(loss.item())

    # 現在の交差エントロピーを出力
    print('Epoch: {}\tCross Entropy: {:.6f}'.format(epoch, sum(losses)))

/usr/local/lib/python3.6/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 0	Cross Entropy: 419.400552


100%|█████████▉| 579756/580000 [01:20<00:00, 9546.73it/s]

Epoch: 1	Cross Entropy: 418.658048
Epoch: 2	Cross Entropy: 416.368108
Epoch: 3	Cross Entropy: 370.661890
Epoch: 4	Cross Entropy: 344.464257
Epoch: 5	Cross Entropy: 334.080223
Epoch: 6	Cross Entropy: 325.701517
Epoch: 7	Cross Entropy: 318.543623
Epoch: 8	Cross Entropy: 309.687193
Epoch: 9	Cross Entropy: 270.733482
Epoch: 10	Cross Entropy: 194.678916
Epoch: 11	Cross Entropy: 157.676627
Epoch: 12	Cross Entropy: 135.690936
Epoch: 13	Cross Entropy: 121.633013
Epoch: 14	Cross Entropy: 109.254712
Epoch: 15	Cross Entropy: 99.319970
Epoch: 16	Cross Entropy: 93.378236
Epoch: 17	Cross Entropy: 83.314507
Epoch: 18	Cross Entropy: 80.174575
Epoch: 19	Cross Entropy: 72.020788


In [10]:
correct = 0
model.eval() # モデルを評価モードに変更
for batch_idx, batch in enumerate(test_iterator):
    texts, labels = batch.text.to(device), batch.label.to(device)
    y = model(texts) # モデルの出力を得る
    result = torch.sigmoid(y) # `TextClassifier`ではsigmoid関数を適用していなかったのでここで適用
    prediction = result >= 0.5 # `result`ベクトルと同じ次元を持ち，`result`の中で0.5以上である次元がTrue，それ以外がFalseであるベクトルを`prediction`とする
    target = labels == 1 # `labels`ベクトルと同じ次元を持ち，`labels`の中で1である次元がTrue，それ以外がFalseであるベクトルを`target`とする
    correct_num = target.eq(prediction).sum().item() # `prediction`ベクトルと`target`ベクトルでTrue/Falseが一致したものの数を数える
    correct += correct_num

# test_iterator.datasetにはテストデータ全体が入っているので，これの長さはテストデータの事例数となる
print("Accuracy: {:.3f}".format(correct / len(test_iterator.dataset)))

/usr/local/lib/python3.6/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


Accuracy: 0.832
